In [1]:
from Shared.specific_CNB_sim import *
import jax.random as random
from scipy.optimize import fsolve
from scipy.optimize import minimize
import shared
import shared_functions
m_l = 0.05
m_h = 0.3 
m_phi = 0.000001

total_start = time.perf_counter()
init_dis =8.178
angle_momentum_decay = jnp.load(f'sim_output/no_gravity/neutrino_angle_momentum_decay.npy') 
decayed_neutrinos_z = jnp.load(f'sim_output/no_gravity/decayed_neutrinos_z.npy')
z_array = jnp.load(f'sim_output/no_gravity/z_int_steps.npy')
neutrino_momenta = jnp.load(f'sim_output/no_gravity/neutrino_momenta.npy')
s_int_steps=jnp.load(f'sim_output/no_gravity/s_int_steps.npy')
z_int_steps=jnp.load(f'sim_output/no_gravity/z_int_steps.npy')

In [ ]:
from Shared.specific_CNB_sim import *

In [ ]:
@jax.jit
def SPEC_n_prefactor(p, z, args):
    '''z and p are both arrays'''
     # Assuming T_CNB is passed as an argument
    return p**2 / (2*jnp.pi**2 * (1+z)**3) * (1 / (jnp.exp(p / (Params.T_CNB * (1+z))) + 1))

In [ ]:
def Xi(z, redshifted_p, args):
    """Auxiliary integral, representing effective time between z_end and z where z is an array."""
    int_steps = 100
    m =Params.m
    # Integrand in xi function
    def integrand(z, redshifted_p,  m):
        p = redshifted_p.T  # necessary operations for broadcasting arrays with different sizes
        z = z.T
        factor_1 = 1 / (CALC_Hubble_rate(z,args) * (1 + z))
        factor_2 = 1 / jnp.sqrt(jnp.square(1 + z[:, :, jnp.newaxis]) + jnp.square(m/ p[:, jnp.newaxis]))
        integrand = factor_1[:, :, jnp.newaxis] * factor_2
        return integrand.T

    # Create redshift integration range.
    z_int_range = jnp.logspace(jnp.log10(z + 1), jnp.log10(Params.z_dec + 1), int_steps) - 1  # integration range for each new z as given in the input variables
    integrand_array = integrand(z_int_range, redshifted_p, Params.m)  # array of integrands in order to integrate in Xi_array
    Xi_array =jax.scipy.integrate.trapezoid(integrand_array, x=z_int_range[jnp.newaxis, :, :], axis=1)
    return Xi_array

In [ ]:
@jax.jit
def CALC_Hubble_rate(z, args):
    # Hubble rate in "numerical" units.
    return Params.H0 * jnp.sqrt(Params.Omega_R * (1 + z)**4 + Params.Omega_M * (1 + z)**3)


In [ ]:
p = jnp.linspace(0.01, 400, 1000) * Params.T_CNB 
z= jnp.array([0, 0.0038223239895366357, 0.007790749585883147, 0.011910861265805747, 0.016188456962865708, 0.02062955622642977, 0.02524040869254421, 0.03002750287859307, 0.03499757531411696, 0.040157620020641244, 0.04551489835385383, 0.05107694922198308, 0.056851599694755006, 0.0628469760178596, 0.06907151504842529, 0.07553397612759427, 0.08224345340690684, 0.08920938864583905, 0.09644158449850457, 0.10395021830821685, 0.11174585642932416, 0.11983946909647203, 0.12824244586221653, 0.1369666116247132, 0.14602424326803673, 0.1554280869385469, 0.16519137598161346, 0.175327849563941, 0.1858517720076989, 0.19677795286366476, 0.2081217677516284, 0.21989917999738326, 0.23212676309675426, 0.244821724038273, 0.25800192751732176, 0.27168592107582, 0.28589296120283125, 0.3006430404328204, 0.3159569154796932, 0.33185613644621015, 0.3483630771498797, 0.36550096660800413, 0.3832939217261868, 0.4017669812362995, 0.4209461409316684, 0.4408583902490648, 0.46153175024897664, 0.48299531304760823, 0.5052792827561012, 0.528415017984583, 0.5524350759708586, 0.5773732583958445, 0.6032646589502149, 0.630145712719202, 0.6580542474550415, 0.687029536809217, 0.7171123555994148, 0.7483450371889591, 0.7807715330594762, 0.8144374746606183, 0.8493902376238842, 0.8856790084309009, 0.9233548536299822, 0.9624707916983682, 1.0030818676512752, 1.0452452305027389, 1.0890202136872689, 1.1344684185554774, 1.1816538010611768, 1.2306427617619466, 1.2815042392598066, 1.3343098072134996, 1.3891337750588981, 1.4460532925792675, 1.5051484584725454, 1.5665024330684194, 1.6302015553538058, 1.69633546447143, 1.7649972258624762, 1.83628346223081, 1.9102944895130896, 1.9871344580460817, 2.066911499129862, 2.1497378771931315, 2.235730147774776, 2.3250093215440115, 2.417701034589889, 2.5139357252198296, 2.613848817515976, 2.7175809119076466, 2.825277983028097, 2.937091585134014, 3.0531790653767907, 3.173703785225734, 3.2988353503547874, 3.4287498493162487, 3.5636301013374028, 3.7036659135887087, 3.849054348285637, 3.9999999999999996])

redshifted_p_range = p[:, np.newaxis] * (1 + z) #using broadcasting methods to multiple z_arr and p_range
xi = Xi(z,redshifted_p_range/(1+z),args=Params())


In [ ]:
#@jax.jit
def n3_p_range(p, z ,args): 
    '''gamma, z_end, m passed on as args'''
    """Number density for m_3 per momentum range, prefactor, xi_factor and decay probability."""
    # Prefactor
    n_pre = SPEC_n_prefactor(p,z,args = Params())

    # Exp-factor with Xi integral.
    Xi_factor = Xi(z, p/(1+z), args = Params())
    n_exp = np.exp(-((Params.m*Params.Gamma*(1+z)* Xi_factor)/p) )
    
    n3_deriv_p_range = n_pre * n_exp  * (-3 / (1 + z)+ 
                  (p / (Params.T_CNB * (1 + z) ** 2)) * 1/(np.exp(p/(Params.T_CNB *(1+z))) + 1) *np.exp(p / (Params.T_CNB  * (1 + z))) - 
                  (Params.Gamma* Params.m/ p)*(1+z) * (Xi_factor/(1+z) -(1/(CALC_Hubble_rate(z,args=Params())))* (1/(np.sqrt((1+z)**2 + (Params.m**2/p**2) )))))

    return n_pre * n_exp, n_pre, Xi_factor, abs(n3_deriv_p_range)


In [ ]:
#@jax.jit
def decay_neutrinos(histogram_data, args):
    total_neutrinos = jnp.sum(histogram_data)

    # Create an array representing individual neutrinos (all ones)
    neutrinos_array =jnp.ones(total_neutrinos)
    key = jax.random.PRNGKey(0)
    # List to store neutrinos_array for each redshift step
    decayed_redshift = []
    #for statement 
    # Iterate over redshift steps
    for i, num_decayed in enumerate(histogram_data):
    
        # Reset decayed indices
        decay_indices = []
        
        # Randomly select neutrinos to decay
        while len(decay_indices) < num_decayed:
            key, subkey = jax.random.split(key)
            index = jax.random.randint(subkey, minval=0, maxval=total_neutrinos, shape=(1,))
            if neutrinos_array[index] == 1:  # Check if neutrino is still active
                decay_indices.append(index)
                neutrinos_array = neutrinos_array.at[index].set(0)
        
        decayed_redshift.append(jnp.copy(neutrinos_array))
      
    return decayed_redshift

def montecarlo(z,p,args):
   # z_array = np.logspace(np.log10(0+1), np.log10(z_dec+1), z_steps)-1 #array of redshifts for testing # neutrino decays at each point
   # np.save('z_array.npy', np.array(z_array))
  
    #distances_arr = redshift_distance(Params.p0, Params.m, z) #translation into distance
    redshifted_p = p[:, None] * (1 + z)  # Broadcasting multiplication

    n3_raw, prefac, xi, decay_prob = n3_p_range(p=redshifted_p, z=z, args=Params())

    n3_deriv_redshifts =jax.scipy.integrate.trapezoid(decay_prob, redshifted_p, axis=0)
    n3_redshifts = jax.scipy.integrate.trapezoid(n3_raw, redshifted_p, axis=0)
    n3_initial = n3_redshifts[-1]
    
    prob_dec = n3_deriv_redshifts / n3_initial
    
    integrated_distribution = n3_redshifts / n3_initial
    
    prob_surv = 1 - prob_dec
    # Generate random numbers using JAX's PRNG
   
     # You can change the seed if needed
    random_numbers = jax.random.uniform(jax.random.PRNGKey(0), shape=(Params.number_neutrinos,))
    occurences_redshifts = z[jnp.searchsorted(integrated_distribution[:-1], random_numbers)]
    histogram_data, bin_edges = np.histogram(np.log10(occurences_redshifts + 1), bins=1000)
    return histogram_data,occurences_redshifts
   # Iterate over redshift steps

    ''' for i, num_decayed in enumerate(histogram_data):
      # Reset decayed indices
       decay_indices = []
       key = jax.random.PRNGKey(i)  # Use redshift step index as a key
      # Randomly select neutrinos to decay
       while len(decay_indices) < num_decayed:
          index = jax.random.randint(key, shape=(), minval=0, maxval=Params.number_neutrinos)
          if neutrinos_array[index] == 1:  # Check if neutrino is still active
              decay_indices.append(index)
              neutrinos_array[index] = 0
      
      # Append a copy of neutrinos_array to the list for each redshift step
       decayed_redshift.append(neutrinos_array.copy())'''

In [ ]:
p = jnp.linspace(0.01, 400, 1000) * Params.T_CNB 
#z= jnp.array([0, 0.0038223239895366357, 0.007790749585883147, 0.011910861265805747, 0.016188456962865708, 0.02062955622642977, 0.02524040869254421, 0.03002750287859307, 0.03499757531411696, 0.040157620020641244, 0.04551489835385383, 0.05107694922198308, 0.056851599694755006, 0.0628469760178596, 0.06907151504842529, 0.07553397612759427, 0.08224345340690684, 0.08920938864583905, 0.09644158449850457, 0.10395021830821685, 0.11174585642932416, 0.11983946909647203, 0.12824244586221653, 0.1369666116247132, 0.14602424326803673, 0.1554280869385469, 0.16519137598161346, 0.175327849563941, 0.1858517720076989, 0.19677795286366476, 0.2081217677516284, 0.21989917999738326, 0.23212676309675426, 0.244821724038273, 0.25800192751732176, 0.27168592107582, 0.28589296120283125, 0.3006430404328204, 0.3159569154796932, 0.33185613644621015, 0.3483630771498797, 0.36550096660800413, 0.3832939217261868, 0.4017669812362995, 0.4209461409316684, 0.4408583902490648, 0.46153175024897664, 0.48299531304760823, 0.5052792827561012, 0.528415017984583, 0.5524350759708586, 0.5773732583958445, 0.6032646589502149, 0.630145712719202, 0.6580542474550415, 0.687029536809217, 0.7171123555994148, 0.7483450371889591, 0.7807715330594762, 0.8144374746606183, 0.8493902376238842, 0.8856790084309009, 0.9233548536299822, 0.9624707916983682, 1.0030818676512752, 1.0452452305027389, 1.0890202136872689, 1.1344684185554774, 1.1816538010611768, 1.2306427617619466, 1.2815042392598066, 1.3343098072134996, 1.3891337750588981, 1.4460532925792675, 1.5051484584725454, 1.5665024330684194, 1.6302015553538058, 1.69633546447143, 1.7649972258624762, 1.83628346223081, 1.9102944895130896, 1.9871344580460817, 2.066911499129862, 2.1497378771931315, 2.235730147774776, 2.3250093215440115, 2.417701034589889, 2.5139357252198296, 2.613848817515976, 2.7175809119076466, 2.825277983028097, 2.937091585134014, 3.0531790653767907, 3.173703785225734, 3.2988353503547874, 3.4287498493162487, 3.5636301013374028, 3.7036659135887087, 3.849054348285637, 3.9999999999999996])
z = np.logspace(np.log10(0+1), np.log10(1e4+1), 1000)-1

histogram_data, occurences_redshifts = montecarlo(z,p,args = Params())

In [11]:
import jax
import jax.numpy as jnp

def update_neutrinos(key_and_array, num_decayed):
    key, neutrinos_array = key_and_array
    # Generate random indices for the decayed neutrinos
    decayed_indices = jax.random.randint(key, shape=(num_decayed,), maxval=neutrinos_array.shape[0])
    # Set the corresponding elements to 0
    
    neutrinos_array = neutrinos_array.at[decayed_indices].set(0)
    return (key, neutrinos_array)

def decay_neutrinos(histogram_data):
    # Generate a random key
    key = jax.random.PRNGKey(0)
    # Initial array with all ones
    neutrinos_array = jnp.ones(jnp.sum(histogram_data))
    # Create a function that updates neutrinos_array based on the histogram data
    update_fn = lambda key_and_array, num_decayed: update_neutrinos(key_and_array, num_decayed)
    # Vectorize the update function
    vmap_update_fn = jax.vmap(update_fn, in_axes=(0, None))
    # Apply the update function for each redshift
    _, final_neutrinos_array = jax.lax.scan(vmap_update_fn, (key, neutrinos_array), histogram_data[1:])
    return final_neutrinos_array

# Example usage:
histogram_data = jnp.array([5, 3, 2])  # Example histogram data
decayed_redshift = decay_neutrinos(histogram_data)
print(decayed_redshift)

ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * one axis had size 2: axis 0 of argument key_and_array[0] of type uint32[2];
  * one axis had size 10: axis 0 of argument key_and_array[1] of type float32[10]

In [22]:
decayed_redshift = decay_neutrinos(histogram_data, args=Params())

In [10]:
print(np.shape(decayed_redshift))

(100, 100000)


In [45]:
import jax
import jax.numpy as jnp
from jax import lax

def decay_neutrinos(histogram_data, args):
    total_neutrinos = jnp.sum(histogram_data)

    # Create an array representing individual neutrinos (all ones)
    neutrinos_array = jnp.ones(total_neutrinos)
    key = jax.random.PRNGKey(0)

    # Generate random indices for all redshift steps at once
    key, *subkeys = jax.random.split(key, len(histogram_data) + 1)
    max_num_decayed = jnp.max(histogram_data)

    random_indices = jax.random.randint(subkeys[0], minval=0, maxval=total_neutrinos, shape=(len(histogram_data), max_num_decayed))
    

    # Function to update neutrinos_array based on decay indices
    def update_neutrinos_array(neutrinos_array, decay_indices):
        print(decay_indices)
        active_indices = jnp.ones(total_neutrinos)
        active_indices = active_indices.at[decay_indices].set(0)
        return active_indices
 
    # Vectorized version of the update_neutrinos_array function
    vmap_update_neutrinos_array = jax.vmap(update_neutrinos_array, in_axes=(None, 0))

    # Apply the decay for each redshift step
    decayed_redshift = vmap_update_neutrinos_array(neutrinos_array, random_indices)
    jnp.save("decayed_redshift.npy", decayed_redshift)
    return decayed_redshift

# Example usage:
histogram_data, occurences_redshifts = montecarlo(z,p,args = Params())  # Example histogram data
args = None  # Example args
decayed_redshift = decay_neutrinos(histogram_data[:50], args)


Traced<ShapedArray(int64[6])>with<BatchTrace(level=1/0)> with
  val = Array([[28, 23, 35, 38, 51, 48],
       [31, 49,  3, 28, 42, 40],
       [ 0, 29,  8, 35, 16, 10],
       [34, 40, 14,  6, 37, 15],
       [16, 28, 34, 27, 54,  8],
       [28, 32, 51, 33, 53, 51],
       [13, 23, 35, 53, 21, 40],
       [28, 18,  3, 21, 26, 42],
       [ 9, 37, 25,  2, 38, 12],
       [ 8,  0, 46, 42, 33, 31],
       [24, 45,  0, 12, 21, 34],
       [31, 38, 43, 16, 25, 17],
       [41, 10,  1, 46, 51, 29],
       [32, 25, 19, 10, 18, 36],
       [ 9, 32, 41, 52, 17, 10],
       [11, 46, 10, 17, 38, 51],
       [ 5,  7, 21, 46, 47, 18],
       [33, 36, 44,  6,  2, 43],
       [ 6,  9, 33, 10, 22, 17],
       [ 3, 32, 30, 25, 12, 36],
       [23, 26,  0, 11, 39, 35],
       [50, 29, 11,  1, 41,  6],
       [32,  3, 49, 40, 51, 54],
       [ 3, 53, 37, 34, 34, 52],
       [14, 10, 41, 27, 20,  7],
       [54, 17, 15, 24,  2, 43],
       [ 2, 12,  3,  4, 12, 17],
       [35, 30, 39, 20,  4, 38],
      

In [ ]:
def find_nearest(array, value):
    idx = jnp.argmin(jnp.abs(array - value))
    return array[idx]

In [ ]:
@jax.jit

def daughter_momentum_4(p_h, angle, m_l, m_phi, m_h):
    E_h = jnp.sqrt(p_h**2 + m_h**2)

    def equation_l(p_l, cos_theta_l, p_h):
        # Calculate E_l and E_h based on the given relation
        E_l = jnp.sqrt(p_l**2 + m_l**2)
        # Calculate cos_theta_daughter using the given equation
        cos_theta_daughter_calculated = (E_h * E_l - (m_h**2 + m_l**2 - m_phi**2) / 2) / (p_h * p_l)
        # Return the difference between the calculated and given values
        return cos_theta_daughter_calculated - cos_theta_l
    
    def equation_phi(p_phi, cos_theta_phi, p_h):
        # Calculate E_phi and E_h based on the given relation
        E_phi = jnp.sqrt(p_phi**2 + m_phi**2)
        # Calculate cos_theta_daughter using the given equation
        cos_theta_daughter_calculated = (E_h * E_phi - (m_h**2 + m_phi**2 - m_l**2) / 2) / (p_h * p_phi)

        # Return the difference between the calculated and given values
        return cos_theta_daughter_calculated - cos_theta_phi
    
    initial_guess = p_h
    solution_l = fsolve(equation_l, initial_guess, args=(jnp.cos(jnp.radians(angle)), p_h))
    solution_phi = fsolve(equation_phi, initial_guess, args=(jnp.cos(jnp.radians(angle)), p_h))
    return jnp.float64(solution_l), jnp.float64(solution_phi)

    

@jax.jit
#specify mass as an argument too
def EOMs_no_gravity(s_val, y, Nr):
    Nr_index = Nr
    
    # Find z corresponding to s via interpolation.
    z =  Utils.jax_interpolate(s_val, s_int_steps, z_int_steps)
    z_nearest = find_nearest(z_array, z)

   # check index of z in our z array
    z_index = jnp.argmax(jnp.equal(z_nearest, z_array)) # Need to access the first element of the array returned by where function
    neutrino_number = jnp.int16(decayed_neutrinos_z[z_index,Nr_index][0])
    prev_neutrino_number = jnp.int16(decayed_neutrinos_z[z_index - 1,Nr_index][0])
    _, u_i_p = y
    
    def true_func(y):
        m_l = 0.05
        m_h = 0.3 
        m_phi = 0.000001
 
        p_i = find_nearest(neutrino_momenta, 0.06 * jnp.linalg.norm(u_i_p, axis=-1)) #neutrino momenta pass as arg 
        p_index = jnp.argmax(jnp.equal(neutrino_momenta, p_i))  # Need to access the first element of the array returned by where function
        T_CNB =0.00016763758644368088
        print(jnp.shape(0.06 * jnp.linalg.norm(u_i_p, axis=-1)))
        angle_decay_theta = random.randint(jax.random.PRNGKey(0),(1,), 0, 179)
        angle_decay_phi = random.randint(jax.random.PRNGKey(0),(1,), 0, 360)
        print(y)
        momentum_daughter = angle_momentum_decay[angle_decay_theta,p_index][0]
        #momentum_daughter_3, momentum_daughter_2 = daughter_momentum_4(jnp.float64(0.06 * jnp.linalg.norm(u_i_p, axis=-1)), angle_decay_theta, m_l, m_phi, m_h)
        
        u_i = jnp.array( [(1 / 0.05) * momentum_daughter * jnp.sin(angle_decay_theta) * jnp.cos(angle_decay_phi),
            (1 / 0.05) * momentum_daughter * jnp.sin(angle_decay_theta) * jnp.sin(angle_decay_phi),
            (1 / 0.05) * momentum_daughter * jnp.cos(angle_decay_theta)])

        return jnp.squeeze(u_i)

    def false_func(y):
        return jnp.squeeze(y[1])

    u_i = jax.lax.cond((neutrino_number == 0) & (prev_neutrino_number == 1), y,  true_func, y, false_func)
  
    dyds = -jnp.array([
        u_i, jnp.zeros(3)
    ])

    return dyds


#@jax.jit
def backtrack_1_neutrino(init_vector, s_int_steps):

    """
    Simulate trajectory of 1 neutrino. Input is 6-dim. vector containing starting positions and velocities of neutrino. Solves ODEs given by the EOMs function with an jax-accelerated integration routine, using the diffrax library. Output are the positions and velocities at each timestep, which was specified with diffrax.SaveAt. 
    """
    y0_r, Nr = init_vector[0:-1], init_vector[-1]
  
    # Initial vector in correct shape for EOMs function
    y0 = y0_r.reshape(2,3)
    
    # ODE solver setup
    term = diffrax.ODETerm(EOMs_no_gravity)
    t0 = s_int_steps[0]
    t1 = s_int_steps[-1]
    dt0 = (s_int_steps[0] + s_int_steps[1]) / 1000
    

    ### ------------- ###
    ### Dopri5 Solver ###
    ### ------------- ###
    solver = diffrax.Dopri5()
    stepsize_controller = diffrax.PIDController(rtol=1e-3, atol=1e-6)
    # note: no change for tighter rtol and atol, e.g. rtol=1e-5, atol=1e-9


    # Specify timesteps where solutions should be saved
    saveat = diffrax.SaveAt(ts=jnp.array(s_int_steps))
    # Solve the coupled ODEs, i.e. the EOMs of the neutrino
    sol = diffrax.diffeqsolve(
        term, solver, 
        t0=t0, t1=t1, dt0=dt0, y0=y0, max_steps=10000,
        saveat=saveat, stepsize_controller=stepsize_controller, args=(int(Nr),))
    
    trajectory = sol.ys.reshape(100,6)

    # Only return the initial [0] and last [-1] positions and velocities
    return jnp.stack([trajectory[0], trajectory[-1]])

#@jax.jit
def simulate_neutrinos_1_pix(init_xyz, init_vels, s_int_steps):
    """
    Function for the multiprocessing routine below, which simulates all neutrinos for 1 pixel on the healpix skymap.
    """

    # Neutrinos per pixel
    nus = init_vels.shape[0]
    # Make vector with same starting position but different velocities
    init_vectors_0 = jnp.array(
        [jnp.concatenate((init_xyz, init_vels[k])) for k in range(nus)])

    Nr_column = jnp.arange(1000).reshape(-1, 1)
    # Concatenate the additional column to the original array
    init_vectors = jnp.hstack((init_vectors_0, Nr_column))

    trajectories = jnp.array([
        backtrack_1_neutrino(vec, s_int_steps) for vec in init_vectors[:5]])

    return trajectories  # shape = (neutrinos, 2, 3)

In [ ]:
# File name ending
end_str = f'halo1'

# Initial position (Earth)
init_xyz = np.array([float(init_dis), 0., 0.])
np.save(f'sim_output/no_gravity/init_xyz_{end_str}.npy', init_xyz)
true_Nr =[]

### ============== ###
### Run Simulation ###
### ============== ###

print(f"*** Simulation for no_gravity ***")

sim_start = time.perf_counter()

with open(f'sim_output/no_gravity/sim_parameters.yaml', 'r') as file:
    sim_setup = yaml.safe_load(file)

init_vels = np.load(f'sim_output/no_gravity/initial_velocities.npy')  
trajectories_1_pix = simulate_neutrinos_1_pix(init_xyz, init_vels[0], s_int_steps)


*** Simulation for no_gravity ***
()
Traced<ShapedArray(float64[2,3])>with<DynamicJaxprTrace(level=5/0)>


In [ ]:
@jax.jit
def daughter_momentum_4(p_h, angle, m_l, m_phi, m_h):
    E_h = jnp.sqrt(p_h**2 + m_h**2)

    def equation_l(p_l, cos_theta_l, p_h):
        # Calculate E_l and E_h based on the given relation
        E_l = jnp.sqrt(p_l**2 + m_l**2)
        # Calculate cos_theta_daughter using the given equation
        cos_theta_daughter_calculated = (E_h * E_l - (m_h**2 + m_l**2 - m_phi**2) / 2) / (p_h * p_l)
        # Return the difference between the calculated and given values
        return cos_theta_daughter_calculated - cos_theta_l
    
    def equation_phi(p_phi, cos_theta_phi, p_h):
        # Calculate E_phi and E_h based on the given relation
        E_phi = jnp.sqrt(p_phi**2 + m_phi**2)
        # Calculate cos_theta_daughter using the given equation
        cos_theta_daughter_calculated = (E_h * E_phi - (m_h**2 + m_phi**2 - m_l**2) / 2) / (p_h * p_phi)

        # Return the difference between the calculated and given values
        return cos_theta_daughter_calculated - cos_theta_phi
    
    initial_guess = p_h
    solution_l = fsolve(equation_l, initial_guess, args=(jnp.cos(jnp.radians(angle)), p_h))
    solution_phi = fsolve(equation_phi, initial_guess, args=(jnp.cos(jnp.radians(angle)), p_h))
    print(np.shape(solution_l))
    print(np.shape(solution_phi))
    return solution_l, solution_phi


In [ ]:
T_CNB =0.00016763758644368088

In [ ]:

def daughter_momentum_4(p_h, angle, m_l, m_phi, m_h):
    E_h = jnp.sqrt(p_h**2 + m_h**2)

    def equation_l(p_l, cos_theta_l, p_h):
        # Calculate E_l and E_h based on the given relation
        E_l = jnp.sqrt(p_l**2 + m_l**2)
        # Calculate cos_theta_daughter using the given equation
        cos_theta_daughter_calculated = (E_h * E_l - (m_h**2 + m_l**2 - m_phi**2) / 2) / (p_h * p_l)
        # Return the difference between the calculated and given values
        return cos_theta_daughter_calculated - cos_theta_l
    
    def equation_phi(p_phi, cos_theta_phi, p_h):
        # Calculate E_phi and E_h based on the given relation
        E_phi = jnp.sqrt(p_phi**2 + m_phi**2)
        # Calculate cos_theta_daughter using the given equation
        cos_theta_daughter_calculated = (E_h * E_phi - (m_h**2 + m_phi**2 - m_l**2) / 2) / (p_h * p_phi)

        # Return the difference between the calculated and given values
        return cos_theta_daughter_calculated - cos_theta_phi
    
    initial_guess = p_h
    solution_l = fsolve(equation_l, initial_guess, args=(jnp.cos(jnp.radians(angle)), p_h))
    solution_phi = fsolve(equation_phi, initial_guess, args=(jnp.cos(jnp.radians(angle)), p_h))
    return solution_l, solution_phi

# Assuming T_CNB and other variables are defined earlier in your code
momentum_daughter, momentum_daughter2 = daughter_momentum_4(0.5 * T_CNB, 180, m_l, m_phi, m_h)


[0.14579027]
